### Partition elements using Unstructured library 

In [ ]:
# It may take longer to install the package
!pip install -qU \
    "unstructured[pdf]==0.12.4" \
    semantic-router

Start by downloading and processing an ArXiv paper.

In [1]:
from unstructured.partition.auto import partition

article_url = "https://arxiv.org/pdf/2402.05131.pdf"
elements = partition(url=article_url, strategy="hi_res", pdf_infer_table_structure=True)

/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Conflict between variables skip_infer_table_types: ['pdf', 'jpg', 'png', 'xls', 'xlsx', 'heic'] and pdf_infer_table_structure: True, please reset skip_infer_table_types to turn on table extraction for PDFs.
This function will be deprecated in a future release and `unstructured` will simply use the DEFAULT_MODEL from `unstructured_inference.model.base` to set default model name
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.c

#### Define helper functions

Validate if parsed title element is a real title

In [2]:
import re


def is_valid_title(title: str) -> bool:
    # Rule 1: Title starts with a lowercase letter
    if re.match(r"^[a-z]", title):
        return False
    # Rule 2: Title has a special character (excluding :, -, and .)
    if re.search(r"[^\w\s:\-\.]", title):
        return False
    # Rule 3: Title ends with a dot
    if title.endswith("."):
        return False
    return True

Group elements by valid titles

In [3]:
from unstructured.documents.elements import Element
from colorama import Fore, Style
from typing import List, Dict


def group_elements_by_title(elements: List[Element]) -> Dict:
    grouped_elements = {}
    current_title = "Untitled"  # Default title for initial text without a title

    for element in elements:
        element_dict = element.to_dict()

        if element_dict.get("type") == "Title":
            potential_title = element_dict.get("text", "Untitled")
            if is_valid_title(potential_title):
                print(f"{Fore.GREEN}{potential_title}: True{Style.RESET_ALL}")
                current_title = potential_title
            else:
                print(f"{Fore.RED}{potential_title}: False{Style.RESET_ALL}")
                continue
        else:
            if current_title not in grouped_elements:
                grouped_elements[current_title] = []
            else:
                grouped_elements[current_title].append(element)
    return grouped_elements

Generates chunks grouped elements using semantic RollingWindow splitter

In [10]:
from semantic_router.splitters import RollingWindowSplitter
from typing import Dict

def create_title_chunks(
    grouped_elements: Dict, splitter: RollingWindowSplitter
) -> list:
    title_with_chunks = []
    for title, elements in grouped_elements.items():
        if not elements:
            continue
        combined_element_texts = []
        chunks = []

        for element in elements:
            if not element.text:
                continue
            element_dict = element.to_dict()
            if element_dict.get("type") == "Table":
                # Process accumulated text before the table
                if combined_element_texts:
                    splits = splitter(combined_element_texts)
                    print("-" * 80)
                    chunks.extend([split.content for split in splits])
                    combined_element_texts = []  # Reset combined texts after processing

                # Add table as a separate chunk
                table_text_html = element.metadata.text_as_html
                chunks.append(table_text_html)
            else:
                combined_element_texts.append(element.text)

        # Process any remaining accumulated text after the last table
        # or if no table was encountered

        if combined_element_texts:
            splits = splitter(combined_element_texts)
            print("-" * 80)
            chunks.extend([split.content for split in splits])

        if chunks:
            title_with_chunks.append({"title": title, "chunks": chunks})

    return title_with_chunks

Display chunked text in colors

In [11]:
from IPython.display import display, HTML
import itertools


def print_chunks_by_title(chunks_by_title):
    color_cycle = itertools.cycle(["red", "green", "blue", "magenta"])
    html_output = ""
    for section in chunks_by_title:
        title = section["title"]
        chunks = section["chunks"]
        html_output += f"<h3 style='color: black;'>{title}</h3>"
        for chunk in chunks:
            color = next(color_cycle)
            html_output += f"<p style='color: {color};'>{chunk}</p>"
    display(HTML(html_output))

### Process the elements

In [15]:
import os
from semantic_router.encoders import OpenAIEncoder

encoder = OpenAIEncoder(openai_api_key=os.environ["OPENAI_API_KEY"])

splitter = RollingWindowSplitter(
    encoder=encoder,
    window_size=1,  # Compares each element with the previous one
    min_split_tokens=1,
    max_split_tokens=500,
    plot_splits=False,
)

In [16]:
grouped_elements = group_elements_by_title(elements)

et! ee: False
b e F 0 1: False
] L C . s c [: False
Financial Report Chunking for Eﬀective Retrieval Augmented Generation: True
Introduction: True
2 Jimeno Yepes et al.: False
1 https://www.sec.gov 2 https://www.sec.gov/files/cf-frm.pdf: False
2 Related work: True
4 Jimeno Yepes et al.: False
3 Methods: True
3.1 RAG setting for the experiments: True
3.2 Indexing and retrieval: True
7 https://weaviate.io/developers/weaviate 8 https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-: False
v1: False
3.3 Generation: True
Question: {query}: False
3.4 Chunking: True
3.5 Dataset: True
4 Results: True
11 https://platform.openai.com/docs/guides/embeddings/limitations-risks: False
10 Jimeno Yepes et al.: False
5 Discussion: True
12 Jimeno Yepes et al.: False
6 Conclusions and Future Work: True
References: True


In [17]:
chunks_by_title = create_title_chunks(grouped_elements, splitter)

/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar

--------------------------------------------------------------------------------


2024-02-26 17:07:33 INFO semantic_router.utils.logger Optimal threshold 0.7912974224053915 found with median tokens (136.5) in target range (1-500).
2024-02-26 17:07:33 INFO semantic_router.utils.logger Splitting Statistics:
  - Total Documents: 3
  - Total Splits: 2
  - Splits by Threshold: 1
  - Splits by Max Chunk Size: 0
  - Last Split: 1
  - Minimum Token Size of Split: 19
  - Maximum Token Size of Split: 254
  - Similarity Split Ratio: 0.50


--------------------------------------------------------------------------------


2024-02-26 17:07:33 INFO semantic_router.utils.logger Optimal threshold 0.8514465425347408 found with median tokens (129.5) in target range (1-500).
2024-02-26 17:07:33 INFO semantic_router.utils.logger Splitting Statistics:
  - Total Documents: 7
  - Total Splits: 4
  - Splits by Threshold: 3
  - Splits by Max Chunk Size: 0
  - Last Split: 1
  - Minimum Token Size of Split: 64
  - Maximum Token Size of Split: 400
  - Similarity Split Ratio: 0.75


...

--------------------------------------------------------------------------------


2024-02-26 17:07:40 INFO semantic_router.utils.logger Optimal threshold 0.8250029487527775 found with median tokens (41.0) in target range (1-500).
2024-02-26 17:07:40 INFO semantic_router.utils.logger Splitting Statistics:
  - Total Documents: 2
  - Total Splits: 1
  - Splits by Threshold: 0
  - Splits by Max Chunk Size: 0
  - Last Split: 1
  - Minimum Token Size of Split: 41
  - Maximum Token Size of Split: 41
  - Similarity Split Ratio: 0.00


--------------------------------------------------------------------------------


2024-02-26 17:07:41 INFO semantic_router.utils.logger Optimal threshold 0.8086076732442027 found with median tokens (108.0) in target range (1-500).
2024-02-26 17:07:41 INFO semantic_router.utils.logger Splitting Statistics:
  - Total Documents: 45
  - Total Splits: 23
  - Splits by Threshold: 22
  - Splits by Max Chunk Size: 0
  - Last Split: 1
  - Minimum Token Size of Split: 4
  - Maximum Token Size of Split: 513
  - Similarity Split Ratio: 0.96


--------------------------------------------------------------------------------


In [18]:
print_chunks_by_title(chunks_by_title)

NarrativeText,"61,780"
Title,"29,664"
ListItem,"33,054"
UncategorizedText,"9,400"
Footer,"1,026"
Table,"7,700"
Header,"3,959"
Image,26
FigureCaption,54
Formula,29
Address,229
